In [ ]:
!pip install git+https://github.com/jpmorganchase/QOKit.git
!pip install qiskit[visualization] qiskit-optimization qiskit-aer
!pip install mqt.bench


  Cloning https://github.com/jpmorganchase/QOKit.git to /tmp/pip-req-build-ol6lpif5
  Running command git clone --filter=blob:none --quiet https://github.com/jpmorganchase/QOKit.git /tmp/pip-req-build-ol6lpif5
  Resolved https://github.com/jpmorganchase/QOKit.git to commit 1cc61ef497d566c40f4b5e0bd0aa1ad0cf880cb4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.4/643.4 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of qiskit-ibm-runtime to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0


### **Q1: Evaluating the Effectiveness of Native Gate Sets Across Quantum Hardware Platforms  [6 points]**


---


Consider the following four quantum hardware platforms, each with its own native gate set:

1. IBM (Superconducting)
2. IonQ (Trapped Ions)
3. Quantinuum (Trapped Ions)

**(a)** Using a **technology-agnostic benchmarking approach**, identify which platform’s native gate set is **most effective** in minimizing **circuit depth** and **gate count** across a range of quantum algorithms and problem sizes. Please follow, the following steps

1. **Generate and compile circuits** for the above benchmarks using the MQT Bench tool:  
* https://github.com/cda-tum/mqt-bench
   

2. **Compile each benchmark circuit** to the native gate set of each platform using MQT bench, use benchmarks with qubits 5,10,15,25, 35, 45.
* https://mqt.readthedocs.io/projects/bench/en/latest/Parameter.html




**(b)** Estimate the SWAP overhead for each benchmark for IonQ Aria 1 (25 Qubits),
Quantinuum H2 (32 Qubits), and  IBM Montreal (27 Qubits). Use the connectivity map in the code window below for the machines.



and use the following table to estimate the execution time for benchmarks with qubits 5,10,15,25, 35. Use qiskit's mapping and routing pass. Note use the connectivity specified below, we have altered for pedagogical purposes.

| Platform           | Native Gates                   | Connectivity       | 1Q Gate Time | 2Q Gate Time | Coherence Time (T₁/T₂)     | Readout Latency |
|--------------------|--------------------------------|--------------------|--------------|--------------|-----------------------------|------------------|
| **IonQ Aria 1**    | Arbitrary 1Q, **XX(θ)**         | Somewhat Fully connected    | 10 µs        | 100 µs       | 1,000 ms / 1,000 ms         | 1 ms             |
| **Quantinuum H2**  | Arbitrary 1Q, **ZZ(θ)**         | Somewhat Fully connected    | 10 µs        | 150 µs       | 500 ms / 500 ms             | 1 ms             |
| **IBM Montreal**   | Rx(π/2), Rz(θ), **CX**          | Heavy-hex (sparse) | 20 ns        | 200 ns       | 100 µs / 100 µs             | 200 µs           |
           |

In [ ]:

Quantinuum_H2_32=[
 (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7),
 (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7),
 (2, 3), (2, 4), (2, 5), (2, 6), (2, 7),
 (3, 4), (3, 5), (3, 6), (3, 7),
 (4, 5), (4, 6), (4, 7),
 (5, 6), (5, 7),
 (6, 7),
 (7, 8),  # inter-group connection

 (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15),
 (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15),
 (10, 11), (10, 12), (10, 13), (10, 14), (10, 15),
 (11, 12), (11, 13), (11, 14), (11, 15),
 (12, 13), (12, 14), (12, 15),
 (13, 14), (13, 15),
 (14, 15),
 (15, 16),  # inter-group connection

 (16, 17), (16, 18), (16, 19), (16, 20), (16, 21), (16, 22), (16, 23),
 (17, 18), (17, 19), (17, 20), (17, 21), (17, 22), (17, 23),
 (18, 19), (18, 20), (18, 21), (18, 22), (18, 23),
 (19, 20), (19, 21), (19, 22), (19, 23),
 (20, 21), (20, 22), (20, 23),
 (21, 22), (21, 23),
 (22, 23),
 (23, 24),  # inter-group connection

 (24, 25), (24, 26), (24, 27), (24, 28), (24, 29), (24, 30), (24, 31),
 (25, 26), (25, 27), (25, 28), (25, 29), (25, 30), (25, 31),
 (26, 27), (26, 28), (26, 29), (26, 30), (26, 31),
 (27, 28), (27, 29), (27, 30), (27, 31),
 (28, 29), (28, 30), (28, 31),
 (29, 30), (29, 31),
 (30, 31)
]


IonQ_25=[
 (0, 1), (0, 2), (0, 3), (0, 4),
 (1, 2), (1, 3), (1, 4),
 (2, 3), (2, 4),
 (3, 4),
 (4, 5),

 (5, 6), (5, 7), (5, 8), (5, 9),
 (6, 7), (6, 8), (6, 9),
 (7, 8), (7, 9),
 (8, 9),
 (9, 10),

 (10, 11), (10, 12), (10, 13), (10, 14),
 (11, 12), (11, 13), (11, 14),
 (12, 13), (12, 14),
 (13, 14),
 (14, 15),

 (15, 16), (15, 17), (15, 18), (15, 19),
 (16, 17), (16, 18), (16, 19),
 (17, 18), (17, 19),
 (18, 19),
 (19, 20),

 (20, 21), (20, 22), (20, 23), (20, 24),
 (21, 22), (21, 23), (21, 24),
 (22, 23), (22, 24),
 (23, 24)
]


edges_IBM_27 = [
    (0, 1), (1, 2), (2, 3),
    (3, 4), (4, 5), (5, 6),
    (6, 7), (7, 8), (8, 9),
    (1, 10), (3, 11), (5, 12), (7, 13), (9, 14),
    (10, 11), (11, 12), (12, 13), (13, 14),
    (10, 15), (11, 16), (12, 17), (13, 18), (14, 19),
    (15, 16), (16, 17), (17, 18), (18, 19),
    (15, 20), (16, 21), (17, 22), (18, 23), (19, 24),
    (20, 21), (21, 22), (22, 23), (23, 24),
    (21, 25), (23, 26)
]





(c) Estimate the fidelity using noisy simulation for the listed benchmarks for 5- and 10-qubit instances per platform. When estimating the fidelity model idling errors. Use equation (9) and (10) from https://arxiv.org/abs/1404.3747 , assume when gates have no operational dependence we can perform them in parallel and there is no crosstalk. Clearly state **Your conclusion:** Which hardware platform is most effective overall, and why?



| Platform           | 1Q Gate Fidelity       | 2Q Gate Fidelity       | Readout Fidelity                                  |
|--------------------|------------------------|-------------------------|----------------------------------------|
| **IonQ Aria 1**    | 99.9%                | 99%                 |  99.99% |
| **Quantinuum H2**  |  99.97%               | 98%                 |    99.99%        |
| **IBM Montreal**   |  99.9%                | 98.5%               | 97%


---

**Benchmarks:**  
Use the following **representative quantum applications** for your evaluations:

1. Amplitude Estimation (AE)  
2. Graph State Preparation (GSP)
3. Portfolio Optimization with QAOA (PoQAOA)  
4. Portfolio Optimization with VQE  (PoVQE)
5. Quantum Approximate Optimization Algorithm (QAOA)  
6. Quantum Fourier Transform (QFT)  
7. Quantum Neural Network (QNN)  
8. Variational Quantum Eigensolver (VQE)  
9. W-State Preparation (W-state)

---

In [ ]:
# check if mqt bench is working
from mqt.bench import get_benchmark

# get a benchmark circuit on algorithmic level representing the GHZ state with 5 qubits
qc_algorithmic_level = get_benchmark(benchmark_name="ghz", level="alg", circuit_size=5)

# draw the circuit
print(qc_algorithmic_level.draw())

                            ┌───┐ ░ ┌─┐            
   q_0: ────────────────────┤ X ├─░─┤M├────────────
                       ┌───┐└─┬─┘ ░ └╥┘┌─┐         
   q_1: ───────────────┤ X ├──■───░──╫─┤M├─────────
                  ┌───┐└─┬─┘      ░  ║ └╥┘┌─┐      
   q_2: ──────────┤ X ├──■────────░──╫──╫─┤M├──────
             ┌───┐└─┬─┘           ░  ║  ║ └╥┘┌─┐   
   q_3: ─────┤ X ├──■─────────────░──╫──╫──╫─┤M├───
        ┌───┐└─┬─┘                ░  ║  ║  ║ └╥┘┌─┐
   q_4: ┤ H ├──■──────────────────░──╫──╫──╫──╫─┤M├
        └───┘                     ░  ║  ║  ║  ║ └╥┘
meas: 5/═════════════════════════════╩══╩══╩══╩══╩═
                                     0  1  2  3  4 


In [ ]:
# Add your code Q1a

ToDO: Analysis

# Q2: Zero Noise Extrapolation [4 points + 1 point]


(a) Use 8-node QAOA with P=2 example from HW-3 Q2 and show that ZNE can significantly improve expected cost estimate. Use the idetical noise configuration as superconducting hardware described in Q1 and connectivity map to compile your circuit and perform ZNE using folded CX gates (local folding) to modulate noise levels. Make sure the additional "CX" gates are introduced after the mapping and routing pass. Use three noise levels -- (1) No gate folding (2) gate folding only on second layer of QAOA (3) gate folding on both layers of QAOA.  



(b) You have three options -- (1) use ZNE on fast but noisy IBM machine (2) use slower but more reliable H2 machine without ZNE or (3) Aria IonQ hardware without ZNE, which is the best option? Backup your claim with numerical simulations.    



(c) Can you do better than (a) without increasing the total execution time ? [Extra Credit]

In [ ]:
# Code for Q2

TODO Q2

# Q3: Unitary Synthesis [3 points]

In this problem, you'll implement a brute force, approximate, single-qubit unitary synthesis algorithm using the Clifford + T gate set. It has been broken down into small parts, so don't be intimidated by the number of subquestions.

## Part A: Hilbert-Schmidt distance

Our algorithm should guarantee the output circuit is close to the target unitary. This distance can be quantified using something called the Hilbert-Schmidt distance, where $n$ is the number of qubits:
$$HS(U_1, U_2) := \sqrt{1-\frac{| Tr(U_1^\dagger U_2)| ^2}{2^{2n}}}$$

This outputs a value between 0 and 1 (inclusive) that indicates how close the two untaries are. 0 means they are equal up to global phase and 1 means they are very not equal.

Is this distance function symmetric? i.e. is $HS(U_1, U_2)$ equal to $HS(U_2, U_1)$? Either prove it is symmetric or provide a counterexample.

Type your answer below:

TODO

## Part B: Implement the Hilbert-Schmidt distance function

Fill in the function below to compute the Hilbert-Schmidt (HS) distance.

In [ ]:
import numpy as np
from numpy.typing import NDArray
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

def get_unitary_from_circuit(circuit: QuantumCircuit):
  return Operator(circuit).data

def hilbert_schmidt(unitary1: NDArray, unitary2: NDArray) -> float:
  pass # TODO: write your code here. You may find some numpy methods to be particularly helpful.

# A test. Do not delete anything below this and make sure the output is included in your submission.
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.s(0)
circuit.t(0)

print(hilbert_schmidt(target_unitary, get_unitary_from_circuit(circuit)))

Is the unitary this circuit implements close to the target unitary?

Type your answer below:

TODO

## Part C: Implement unitary synthesis

Implement a function to synthesize a quantum circuit that is within epsilon of the target unitary, as measured by the HS distance.

Fill in the function below.

In [ ]:
# Your function should return a circuit with only these gates
cliffordt_gate_set_1q = ["h", "s", "t"]

"""
target_unitary: the unitary to implement
epsilon: a float between 0 and 1 (inclusive)

Returns a qiskit.QuantumCircuit object such that hilbert_schmidt(target_unitary, get_unitary_from_circuit(synthesize(target_unitary, epsilon))) <= epsilon is True.
"""
def synthesize(target_unitary: np.NDArray, epsilon: float) -> QuantumCircuit:
  # Hint: brute force search by enumerating circuits starting from the empty circuit, then all circuits with 1 gate, and so on.
  pass

# A test. Do not delete anything below this and make sure the output is included in your submission.
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
synthesized_circuit = synthesize(target_unitary, 0.5)
print(synthesized_circuit)
print(hilbert_schmidt(target_unitary, get_unitary_from_circuit(synthesized_circuit)) <= 0.5)

## Part D: Scalability

Sweep the following epsilon values and plot the time in seconds it takes to find a solution.

In [ ]:
target_unitary = np.matrix([[-0.63093663-0.52401339j,  0.2720076 +0.50332971j],
       [ 0.42561408+0.38233701j,  0.36164925+0.73612559j]])
epsilons = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

# TODO: write your code here and make sure the plot is included in the ouput

What does the plot tell you about how this algorithm scales?

Type your answer below:

TODO

## Part E: Why is unitary synthesis important?

Why do we even need to solve this problem? What is the significance of the Clifford + T gate set and what type of gate does it fail to support natively?

Type your answer below:

TODO

# Q4: Gridsynth [2 Points]
Gridsynth is the state-of-the-art (ancilla-free, deterministic) synthesis algorithm to approximate Rz gates with Clifford + T. Gridsynth leverages insights from number theory to achieve optimal or near-optimal approximations for any desired error threshold $\epsilon$. By optimal, we mean the saturation of the T count lower bound given an $\epsilon$, which you will explore in Part A.

In [ ]:
!pip install pygridsynth numpy scipy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.2 MB/s eta 0:00:00


## Part A
To approximate an Rz to error $\epsilon$ (gridsynth uses the operator norm as the error metric, which is very close to the distance metric we used in Q3), we need at least $a\log_2 \frac{1}{\epsilon}+b$ T gates in the typical case, where $a$ and $b$ are two real numbers. In this part, you will use a Python implementation of gridsynth to synthesize random Rz gates at different $\epsilon$ to fit $a$ and $b$.

In [ ]:
from pygridsynth import gridsynth_gates
import numpy as np

epsilons = [10**(-i) for i in range(2, 8)]
num_angles = 100
angles = np.random.default_rng(42).uniform(0, 2 * np.pi, size=num_angles)
t_counts = ... # should be of shape num_epsilons * num_angles
# todo: fill in the T counts for all epsilon and angle pairs

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def fitting_func(x, a, b):
    # todo

# here we fit all data points (instead of the worst points / "lower bound") for a more accurate estimation in Part B
popt, pcov = curve_fit(fitting_func, np.repeat(epsilons, num_angles), np.array(t_counts).reshape(-1), p0=(1, 0))
print(f"a = {popt[0]}  b = {popt[1]}")
plt.scatter(..., s=1, label="data", c="tab:blue") # todo: scatter plot of T count data points
plt.semilogx(..., label="fit", color="tab:orange") # todo: plot the fitted function
plt.xlabel("Epsilon")
plt.ylabel("T Count")
plt.legend()

## Part B Overhead of FTQC
Now let's estimate the number of T gates needed for executing a QAOA circuit fault-tolerantly. Since the only source of non-Cliffordness in QAOA is the axial rotations, we first count the number of $R_z$ ($R_x = HR_zH$) gates we need to synthesize in a 10-layer QAOA solving a MaxCut problem with a 100-node 3-regular graph. Then we multiply the number of $R_z$ gates by the rough number of $T$ gates we estimated in Part A for a given $\epsilon$.

> Hint: In a 3-regular graph, each node has 3 edges. How many edges are in the graph? How many Pauli ZZ terms are in the Hamiltonian? How many $R_z$ gates are needed for each ZZ term?

This is a hand-derivation math problem, but feel free to get inspirations or verify your findings by coding.

#### (1) What is the number of $R_z$ gates we need to synthesize in a 10-layer QAOA solving a MaxCut problem with a 100-node 3-regular graph? Remember the $R_x$ gates also need to be transpiled to $R_z$ gates to synthesize.

**Solution:**

Todo

#### (2) To set the synthesis error threshold $\epsilon$, we use a very naive approach here that simply divides the circuit-level error threshold by the number of gates to be synthesized. For a circuit-level error threshold of 0.01, what is the $\epsilon$ for each individual $R_z$ gate? What is the corresponding T gate count needed in total?

> Remark: In error correction codes, (logical) T gates are typically hundreds of times more costly to implement compared to physical gates. This should give you an idea of the overhead of FTQC.

**Solution:**

Todo